In [1]:
using Pkg
Pkg.activate("lab3")
Pkg.add("GLPKMathProgInterface")
Pkg.add(url="https://github.com/sisl/NeuralVerification.jl")
Pkg.add("LazySets")
Pkg.add("IntervalArithmetic")
Pkg.add("Interpolations")
Pkg.status()

  Activating project at `~/verification-neural-networks/NeuralVerification-CARS-Workshop-master/lab3`
   Resolving package versions...
  No Changes to `~/verification-neural-networks/NeuralVerification-CARS-Workshop-master/lab3/Project.toml`
  No Changes to `~/verification-neural-networks/NeuralVerification-CARS-Workshop-master/lab3/Manifest.toml`
    Updating git-repo `https://github.com/sisl/NeuralVerification.jl`
   Resolving package versions...
  No Changes to `~/verification-neural-networks/NeuralVerification-CARS-Workshop-master/lab3/Project.toml`
  No Changes to `~/verification-neural-networks/NeuralVerification-CARS-Workshop-master/lab3/Manifest.toml`
   Resolving package versions...
  No Changes to `~/verification-neural-networks/NeuralVerification-CARS-Workshop-master/lab3/Project.toml`
  No Changes to `~/verification-neural-networks/NeuralVerification-CARS-Workshop-master/lab3/Manifest.toml`
   Resolving package versions...
  No Changes to `~/verification-neural-networks/Neu

Status `~/verification-neural-networks/NeuralVerification-CARS-Workshop-master/lab3/Project.toml`
  [3c7084bd] GLPKMathProgInterface v0.5.0
  [a98d9a8b] Interpolations v0.15.1
⌅ [d1acc4aa] IntervalArithmetic v0.21.2
⌃ [b4f0291d] LazySets v2.14.2
  [146f25fa] NeuralVerification v0.1.0 `https://github.com/sisl/NeuralVerification.jl#master`
  [91a5bcdd] Plots v1.40.9
  [d330b81b] PyPlot v2.11.5
Info Packages marked with ⌃ and ⌅ have new versions available, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [2]:
using LazySets, IntervalArithmetic
using Interpolations # pour activation
using NeuralVerification
import NeuralVerification: Network, Layer, ReLU, Id, compute_output, ActivationFunction, get_bounds, read_nnet, affine_map, overapproximate, approximate_act_map

# Reading .nnet file

In [3]:
# include("utils/activation.jl")
# include("utils/network.jl")
# include("utils/problem.jl")
# include("utils/util.jl")

# Slightly modidied from utils/network.jl
# To be modified for interpretation in zonotopes
function interval_approximate_act_map(act::ActivationFunction, input::Hyperrectangle)
    b    = act.(input.center)
    bmax = act.(high(input))
    bmin = act.(low(input))
    c    = (bmax + bmin)/2
    r    = (bmax - bmin)/2
    return Hyperrectangle(c, r)
end

interval_approximate_act_map(layer::Layer, input::Hyperrectangle) = approximate_act_map(layer.activation, input)

# Slightly modidied from utils/network.jl
# To be modified for better interpretation in zonotopes (it uses interval outer-approximation here)
function interval_approximate_affine_map(layer::Layer, input)
    input = overapproximate(input, Hyperrectangle)
    c = NeuralVerification.affine_map(layer, input.center)
    r = abs.(layer.weights) * input.radius
    return Hyperrectangle(c, r)
end

function net_bounds(nnet::Network, input)
    bounds = Vector{Hyperrectangle}(undef, length(nnet.layers) + 1)
    bounds[1] = input
    b = input
    for (i, layer) in enumerate(nnet.layers)
       b = interval_approximate_affine_map(layer, bounds[i])
       bounds[i+1] = interval_approximate_act_map(layer, b)
    end
    return bounds
end


net_bounds (generic function with 1 method)

In [4]:
print("Reading ")
println("single.nnet")

nnet = read_nnet("single.nnet")

# determine which is the last layer P
P = length(nnet.layers)
print("Number of layers=")
println(P)

# N is the number of input neurons (layers[1])
N = length(nnet.layers[1].weights)÷length(nnet.layers[1].bias)
print("Input size= ")
println(N)

# Last layer size
M = length((nnet.layers[P]).bias)
print("Output size= ")
println(M)

# print layer by layer structure
for (i, layer) in enumerate(nnet.layers)
   print("Layer ")
   print(i)
   print(" has size ")
   println(length(layer.weights)÷length(layer.bias))
end

# construct input hyperrectangle [-1,1]^N
input = Hyperrectangle(zeros(N),ones(N))

# call for interval abstraction
bounds = net_bounds(nnet, input)

# print result
   print("Output neurons values")
   print(" in ")
   println(bounds[P+1])

Reading single.nnet
Number of layers=2
Input size= 2
Output size= 2
Layer 1 has size 2
Layer 2 has size 2
Output neurons values in Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}([0.5, 1.5], [0.5, 1.5])


# Single-layer analysis

## Zone

In [5]:
mutable struct Zone #<: Lazyset
    numinp::Int			
    numvars::Int
    DBM::Matrix{Float64}
end

In [6]:
exzone = Zone(1, 1, [0 1 1; 1 0 1; 1 1 0])

Zone(1, 1, [0.0 1.0 1.0; 1.0 0.0 1.0; 1.0 1.0 0.0])

## Computing the best enclosing zone for a linear layer

In [9]:
function zone_approximate_affine_map(layer::Layer, input::Hyperrectangle)
    # Get the number of input and output neurons
    num_inp = length(input.center) # number of input neurons
    num_out = length(layer.bias)
    
    # Initialize the DBM matrix for the zone
    DBM = zeros(Float64, num_out + 1, num_out + 1)
    
    # Compute the center and radius of the input hyperrectangle
    c_in = input.center
    r_in = input.radius
    
    # Compute the center and radius of the output hyperrectangle
    c_out = layer.weights * c_in .+ layer.bias
    r_out = abs.(layer.weights) * r_in
    
    # Fill the DBM matrix
    for i in 1:num_out
        DBM[i+1, 1] = c_out[i] - r_out[i]
        DBM[1, i+1] = c_out[i] + r_out[i]
        for j in 1:num_out
            if i != j
                DBM[i+1, j+1] = r_out[i] + r_out[j]
            end
        end
    end
    
    return Zone(num_inp, num_out, DBM)
end

zone_approximate_affine_map (generic function with 1 method)

## Closure of DBMs

In [10]:
function zone_closure(input::Zone)
    num_vars = input.numvars
    DBM = copy(input.DBM)
    
    for k in 1:num_vars+1
        for i in 1:num_vars+1
            for j in 1:num_vars+1
                DBM[i, j] = min(DBM[i, j], DBM[i, k] + DBM[k, j])
            end
        end
    end
    
    return Zone(input.numinp, num_vars, DBM)
end

zone_closure (generic function with 1 method)

## Interpreting the ReLU function on zones

In [12]:
function zone_approximate_act_map(act::ActivationFunction, input::Zone)
    if act == Id()
        return input
    else # ReLU case
        db = copy(input.DBM)
        num_vars = input.numvars
        
        for i in 1:num_vars
            db[i+1, 1] = max(0, db[i+1, 1])
            db[1, i+1] = max(0, db[1, i+1])
            for j in 1:num_vars
                if i != j
                    db[i+1, j+1] = max(0, db[i+1, j+1])
                end
            end
        end
        
        return Zone(input.numinp, num_vars, db)
    end
end

zone_approximate_act_map (generic function with 1 method)

## Finding the enclosing hypercube of a zone

In [13]:
function interval_approximate_zone(input::Zone)
    # Perform the closure of the DBM
    closed_zone = zone_closure(input)
    DBM = closed_zone.DBM
    num_vars = closed_zone.numvars
    
    # Initialize the center and radius of the hyperrectangle
    center = zeros(Float64, num_vars)
    radius = zeros(Float64, num_vars)
    
    # Compute the center and radius for each variable
    for i in 1:num_vars
        lower_bound = DBM[i+1, 1]
        upper_bound = DBM[1, i+1]
        center[i] = (upper_bound + lower_bound) / 2
        radius[i] = (upper_bound - lower_bound) / 2
    end
    
    return Hyperrectangle(center, radius)
end

interval_approximate_zone (generic function with 1 method)

# Comparison with other analyzes

# To go further (project)